In [2]:

import numpy as np
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
from netCDF4 import num2date
import datetime

from tqdm import tqdm

tqdm.pandas()
from shapely.geometry import Point

import matplotlib.pyplot as plt

import platform

from cartopy import crs as ccrs

if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

In [3]:
nc_data = nc.Dataset("cru_ts4.05.1901.2020.tmp.dat.nc")
raw_time_data = np.array(nc_data.variables['time'])
raw_tmp_data = np.array(nc_data.variables['tmp'])

<ipython-input-3-3e0e9f740443>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_time_data = np.array(nc_data.variables['time'])
<ipython-input-3-3e0e9f740443>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_tmp_data = np.array(nc_data.variables['tmp'])


In [4]:
def cftime2datetime(cftime, units, format='%Y-%m-%d %H:%M:%S'):
    """
    将nc文件里面的时间格式 从cftime 转换到 datetime格式
    :param cftime:
    :param units:
    :param format:
    :return:
    """
    return datetime.datetime.strptime(num2date(times=cftime, units=units).strftime(format), format)


clean_time_data = pd.Series([cftime2datetime(i, units='days since 1900-1-1') for i in raw_time_data])
clean_time_data[:4]

0   1901-01-16
1   1901-02-15
2   1901-03-16
3   1901-04-16
dtype: datetime64[ns]

In [6]:
clean_time_data.dt.year == 2020

0       False
1       False
2       False
3       False
4       False
        ...  
1435     True
1436     True
1437     True
1438     True
1439     True
Length: 1440, dtype: bool

In [10]:
raw_tmp_data[clean_time_data.dt.year == 2020, :, :].shape

(12, 360, 720)

In [14]:
raw_tmp_data[(1901 <= clean_time_data.dt.year) & (clean_time_data.dt.year <= 2012), :, :].shape

(1344, 360, 720)

In [16]:
tmp_missing_value = nc_data.variables['tmp'].missing_value

test = raw_tmp_data[clean_time_data == '1901-01', :, :]
test[test == tmp_missing_value] = np.nan

test = np.nanmean(test, axis=0)
test.shape

<ipython-input-16-4b841cd50c7d>:6: RuntimeWarning: Mean of empty slice
  test = np.nanmean(test, axis=0)


(360, 720)

In [19]:
test = raw_tmp_data[('1901-01' <=clean_time_data) & (clean_time_data <= '1902-02') , :, :]
test.shape

(13, 360, 720)

In [ ]:
test[test == tmp_missing_value] = np.nan

test = np.nanmean(test, axis=0)
test.shape

In [18]:
clean_time_data

0      1901-01-16
1      1901-02-15
2      1901-03-16
3      1901-04-16
4      1901-05-16
          ...    
1435   2020-08-16
1436   2020-09-16
1437   2020-10-16
1438   2020-11-16
1439   2020-12-16
Length: 1440, dtype: datetime64[ns]

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr